In [1]:
!nvidia-smi

Fri Jul 31 07:40:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 0. 설치

In [2]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 710.5MB 27kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 256kB 7.4MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470030 sha256=98340f8fbe4c190d399414f4a4983d6d18d3cf503e35f6e4a15d9b95c15ed3c6
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 7.3MB/s 
     |████████████████████████████████| 317kB 7.1MB/s 
     |████████████████████████████████| 890kB 19.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=dafddc0cf6fd307f0804a43270e686c63527a31f493be58d2dd7ee1ca4d0d0c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremo

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-b4zbb2yo
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-b4zbb2yo
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12871 sha256=c0597bacb211033e6e730b2bd47a78d8d9ecada5fc16eddf40476d77d67c9392
  Stored in directory: /tmp/pip-ephem-wheel-cache-9vvepilx/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


# 1. 불러오기

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [9]:
device = torch.device('cuda:0')
print(device)

cuda:0


In [10]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


## 데이터 불러오기

In [88]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-07-31 09:16:50--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-07-31 09:16:50--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8f8fb4ee6e4c8e780e78fb6c67.dl.dropboxusercontent.com/cd/0/get/A8hT-lOrxt344NpZ6R0V9r75XgS2KDb_sLkCPppMW5mr8WWHTxpRIDQeFBY_jqP8-Yr5Uk_X5np4hFzQl18XzI68v33dmQxHFFfyZ0sCKbd2JNMc7gMZvm6F9k0myS2FxrY/file?dl=1# [following]
--2020-07-31 09:16:50--  https://uc8f8fb4ee6e4c8e780e78fb6c67.dl.dropboxusercontent.com/cd/0/get/A8hT-lOrxt344NpZ6R0V9r75XgS2KDb_sLkCPppMW5mr8WWHTxpRIDQeFBY_jqP8-Yr5Uk_X5np4hFzQl18XzI68v33dmQ

In [89]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [91]:
dataset_train[0:3]

[['아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['너무재밓었다그래서보는것을추천한다', '0']]

## 데이터 가공

In [75]:
tokenizer = get_tokenizer() # KoBert tokenizer
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [42]:
trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pad=True, pair=False)

- 아래의 값을 보면 패딩이 1로 들어가는 것을 알 수 있다

In [93]:
trans([dataset_train[0][0]])

(array([   2, 3093, 1698, 6456,  517,   54,  517,   54, 4368, 4396, 7316,
        5655, 5703, 2073,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32))

- `(token_ids, valid_length, segment_ids, label) = trans([dataset_train[0][0]])` 을 의미한다.

In [86]:
# Setting hyperparameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [129]:
# dataloader에 넣기 위해 dataset 만들기
class KobertDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [111]:
# train 시간이 너무 오래 걸려서 일부 데이터만 사용했다
sampleTrain = dataset_train[:50000]
sampleTest = dataset_test[:5000]

# Dataset
KobertTrain = KobertDataset(sampleTrain, 0, 1, tok, max_len, True, False)
KobertTest = KobertDataset(sampleTest, 0, 1, tok, max_len, True, False)

# DataLoader
trainDataloader = DataLoader(KobertTrain, batch_size=batch_size, num_workers=5)
testDataloader = DataLoader(KobertTest, batch_size=batch_size, num_workers=5)

# 2. Model

In [113]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hiddenSize = 768,
                 numClasses=2,
                 drRate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        
        self.bert = bert
        self.drRate = drRate
                 
        self.classifier = nn.Linear(hiddenSize , numClasses)
        if drRate:
            self.dropout = nn.Dropout(p=drRate)
    
    def gen_attention_mask(self, tokenIds, validLength): 
        attentionMask = torch.zeros_like(tokenIds)
        for i, v in enumerate(validLength):
            attentionMask[i][:v] = 1
        return attentionMask.float()

    def forward(self, tokenIds, validLength, segmentIds):
        attentionMask = self.gen_attention_mask(tokenIds, validLength)
    
        _, pooler = self.bert(input_ids = tokenIds, token_type_ids = segmentIds.long(), attention_mask = attentionMask.float().to(tokenIds.device))
        if self.drRate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [114]:
model = BERTClassifier(bertmodel,  drRate=0.5).to(device)

In [115]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [116]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(trainDataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [118]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [119]:
import random

In [120]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(trainDataloader)):
        optimizer.zero_grad()

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # test
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(testDataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


epoch 1 batch id 1 loss 0.723616898059845 train acc 0.515625
epoch 1 batch id 201 loss 0.452615886926651 train acc 0.6517412935323383
epoch 1 batch id 401 loss 0.42734646797180176 train acc 0.736673940149626
epoch 1 batch id 601 loss 0.4007650911808014 train acc 0.7741524542429284

epoch 1 train acc 0.7941975703324808


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8690664556962026


epoch 2 batch id 1 loss 0.4798983335494995 train acc 0.796875
epoch 2 batch id 201 loss 0.24901050329208374 train acc 0.8645055970149254
epoch 2 batch id 401 loss 0.29674187302589417 train acc 0.8774937655860349
epoch 2 batch id 601 loss 0.4027092754840851 train acc 0.8869332362728786

epoch 2 train acc 0.8926430626598465


...


epoch 3 test acc 0.8852848101265823


epoch 4 batch id 1 loss 0.22941581904888153 train acc 0.9375
epoch 4 batch id 201 loss 0.16536200046539307 train acc 0.9539800995024875
epoch 4 batch id 401 loss 0.15123321115970612 train acc 0.9577618453865336
epoch 4 batch id 601 loss 0.22736486792564392 train acc 0.9605605241264559

epoch 4 train acc 0.9631353900255755



epoch 4 test acc 0.8833069620253164


epoch 5 batch id 1 loss 0.214426651597023 train acc 0.90625
epoch 5 batch id 201 loss 0.15409280359745026 train acc 0.9744247512437811
epoch 5 batch id 401 loss 0.03396403044462204 train acc 0.9774002493765586
epoch 5 batch id 601 loss 0.12070764601230621 train acc 0.9781613976705491

epoch 5 train acc 0.9789601982097187



epoch 5 test acc 0.8902294303797469


- 대략 90분정도 걸린 것 같다
- 더 많은 데이터, 긴 시간동안 훈련시키면 더 좋은 모델이 나올 것이다.
    - 훈련 정확도 : 0.98
    - 테스트 정확도 : 0.89

# new data test

In [122]:
# 모델 저장
torch.save(model.state_dict(), 'model')

In [133]:
class tmpDataset(Dataset):
    def __init__(self, line, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([line[sent_idx]])]
        self.labels = [np.int32(line[label_idx])]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [150]:
# 넣고 싶은 댓글과 긍부정 정답을 같이 넣어준다
line = ['최고의 영화라고 할 수 있다. 2편 나오면 무조건 본다.','1']
tmp = tmpDataset(line, 0, 1, tok, max_len, True, False)
tmpLoader = DataLoader(tmp, batch_size=1)

tmpIter = iter(tmpLoader)
(token_ids, valid_length, segment_ids, label) = tmpIter.next()

token_ids = token_ids.long().to(device)
segment_ids = segment_ids.long().to(device)
valid_length= valid_length
# label = label.long().to(device)

out = model(token_ids, valid_length, segment_ids)
logits = out[0]
logits = logits.detach().cpu().numpy()

print(line)
print(logits)
print(f'model predict : {np.argmax(logits)}')
if label == np.argmax(logits):
    print('model correct!')
    if np.argmax(logits) == 0 :
        print('이 댓글은 부정적입니다.')
    else :
        print("이 댓글은 긍정적입니다.")
else :
    print('model fails...')


['최고의 영화라고 할 수 있다. 2편 나오면 무조건 본다.', '1']
[-2.8754344  2.6775577]
model predict : 1
model correct!
이 댓글은 긍정적입니다.
